In [4]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [16]:
def load_csv_data(data_path, train_path):
    '''
    读取csv文件
    '''
    all_data = pd.read_csv(data_path).drop(['Unnamed: 0'], axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
    train_id = pd.read_pickle(train_path)
    df = all_data.join(train_id.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
    return df

In [17]:
data_path = '../data_20220831/raw_cvjd_20220831_spark.csv'
train_path = '../generalization_data/cvjd_train_filter_54339.pkl'
load_csv_data(data_path, train_path) 

/Users/jinrirencai/opt/anaconda3/envs/tsinghua/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [19]:
train_data_lsa_cvjd_54339 = pd.read_pickle('../generalization_data/train_data_lsa_cvjd_54339.pkl')


In [20]:
train_data_lsa_cvjd_54339[['cv_id', 'jd_id']]

,cv_id,jd_id
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f
...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080
